In [2]:
!pip install transformers==4.19.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 63.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [3]:
"""
Code to create distilbert model. Predicting distinctiveness indicator
"""

# Import Libraries
import sys
sys.path.append('../../python_packages/')
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW
from tqdm import tqdm, trange
import torch
import torch.nn as nn
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from time import time
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device available: ",device)

Device available:  cuda


In [4]:
# Input
data_path = '/content/drive/MyDrive/NLP_LSS/distinctiveness_indicator/distilbert/data/df_distilbert_input_unprocessed.pkl'

# DistilBERT
dbert_model_path = '/content/drive/MyDrive/NLP_LSS/distinctiveness_indicator/distilbert/model/distilbert_unprocessed_wo_mark.pth'


In [5]:
# Load DistillBERT model
model_name = 'distilbert-base-cased'
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
optimizer = torch.optim.Adam([
    {'params': model.distilbert.parameters(), 'lr': 1e-5},
    {'params': model.classifier.parameters(), 'lr': 1e-3}
])

# Read data
df = pd.read_pickle(data_path)
df['filing_dt'] = pd.to_datetime(df['filing_dt'])

# Incidence Rate
print("Incidence Rate: ", ((100*df['distinct_ind'].sum())/(df.shape[0])) )

Downloading:   0%|          | 0.00/465 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/251M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Incidence Rate:  87.19046356698402


In [6]:
# Create bert input w/o mark
df['bert_input_unprocessed_wo_mark'] = '[CLS] ' + df['statement_unprocessed'] + ' [SEP] ' + df['mark_translated'] + ' [SEP] ' + df['wordnet_text'] + ' [SEP] ' + df['mark_length_text'] + ' [SEP] ' + df['nice_cat_text'] + ' [SEP] ' + df['nice_description_unpreprocessed'] + ' [SEP] ' + df['pseudo_mark_unprocessed']
print(df['bert_input_unprocessed_wo_mark'][10])
df.head()

[CLS] Providing news and information in the field of current events relating to software applications [SEP] no translation required [SEP] mark absent in Wordnet [SEP] mark length is 1 [SEP] NICE category is 41 [SEP] Education; Providing of training; Entertainment; Sporting and cultural activities [SEP] Pseudo mark is APPLICATIONS WEEK


,serial_no,filing_dt,mark_processed,mark_unprocessed,distinct_ind,intl_class_cd,mark_len,wn_ind,mark_final,translated_ind,...,pseudo_mark_processed,pseudo_mark_unprocessed,statement_processed,statement_unprocessed,wordnet_text,mark_length_text,nice_cat_text,bert_input_processed,bert_input_unprocessed,bert_input_unprocessed_wo_mark
0,85507018,2012-01-01,RODOPTICS,RODOPTICS,1,9,1,0,RODOPTICS,0.0,...,Pseudo mark is rod optics,Pseudo mark is ROD OPTICS,eyeglasses eyewear namely eyeglasses reading g...,"Eyeglasses; Eyewear, namely, eyeglasses, readi...",mark absent in Wordnet,mark length is 1,NICE category is 9,[CLS] RODOPTICS [SEP] eyeglasses eyewear namel...,"[CLS] RODOPTICS [SEP] Eyeglasses; Eyewear, nam...","[CLS] Eyeglasses; Eyewear, namely, eyeglasses,..."
1,85507042,2012-01-01,TRIVEDI WINE,TRIVEDI WINE,0,35,2,1,TRIVEDI WINE,0.0,...,no Pseudo mark,no Pseudo mark,intent to use descriptor trivedi wine in conne...,"Intent to use descriptor ""TRIVEDI WINE"" in con...",mark present in Wordnet,mark length is 2,NICE category is 35,[CLS] TRIVEDI WINE [SEP] intent to use descrip...,[CLS] TRIVEDI WINE [SEP] Intent to use descrip...,"[CLS] Intent to use descriptor ""TRIVEDI WINE"" ..."
2,85507043,2012-01-01,101 GLASSES,101 GLASSES,1,9,2,1,101 GLASSES,0.0,...,no Pseudo mark,no Pseudo mark,glasses eyewear namely sunglasses eyeglasses o...,"""GLASSES"" Eyewear, namely, sunglasses, eyeglas...",mark present in Wordnet,mark length is 2,NICE category is 9,[CLS] 101 GLASSES [SEP] glasses eyewear namely...,"[CLS] 101 GLASSES [SEP] ""GLASSES"" Eyewear, nam...","[CLS] ""GLASSES"" Eyewear, namely, sunglasses, e..."
3,85507044,2012-01-01,MILLIONAIRE TREASURE HUNT,MILLIONAIRE TREASURE HUNT,1,41,3,1,MILLIONAIRE TREASURE HUNT,0.0,...,no Pseudo mark,no Pseudo mark,entertainment services in the nature of an ong...,Entertainment services in the nature of an on-...,mark present in Wordnet,mark length is 3,NICE category is 41,[CLS] MILLIONAIRE TREASURE HUNT [SEP] entertai...,[CLS] MILLIONAIRE TREASURE HUNT [SEP] Entertai...,[CLS] Entertainment services in the nature of ...
4,85507045,2012-01-01,STONE WIND AND VINES,"STONE, WIND AND VINES",1,33,4,1,STONE WIND AND VINES,0.0,...,no Pseudo mark,no Pseudo mark,wine,Wine,mark present in Wordnet,mark length is 4,NICE category is 33,[CLS] STONE WIND AND VINES [SEP] wine [SEP] no...,"[CLS] STONE, WIND AND VINES [SEP] Wine [SEP] n...",[CLS] Wine [SEP] no translation required [SEP]...


In [7]:
# Divide data into train, test and validation
df_train = df[(df['filing_dt']>=pd.to_datetime('2012-01-01')) & (df['filing_dt']<=pd.to_datetime('2017-12-31'))]
df_val = df[(df['filing_dt']>=pd.to_datetime('2018-01-01')) & (df['filing_dt']<=pd.to_datetime('2018-12-31'))]
df_test = df[(df['filing_dt']>=pd.to_datetime('2019-01-01')) & (df['filing_dt']<=pd.to_datetime('2019-12-31'))]
print("Train data shape: ", df_train.shape)
print("Validation data shape: ", df_val.shape)
print("Test data shape: ", df_test.shape)


Train data shape:  (1001779, 23)
Validation data shape:  (234752, 23)
Test data shape:  (264827, 23)


In [8]:
# Create X and Y for modelling
X_train = np.array(df_train['bert_input_unprocessed_wo_mark'])
y_train = np.array(df_train['distinct_ind'])
X_val = np.array(df_val['bert_input_unprocessed_wo_mark'])
y_val = np.array(df_val['distinct_ind'])
X_test = np.array(df_test['bert_input_unprocessed_wo_mark'])
y_test = np.array(df_test['distinct_ind'])

# Generate Batches
batch_size = 16
train_max_idx = batch_size * (len(X_train)//batch_size)
val_max_idx = batch_size * (len(X_val)//batch_size)
test_max_idx = batch_size * (len(X_test)//batch_size)

X_train = X_train[:train_max_idx]
X_train = X_train.reshape(-1, batch_size)
X_train = X_train.tolist()
y_train = y_train[:train_max_idx]
y_train = y_train.reshape(-1, batch_size)

X_val = X_val[:val_max_idx]
X_val = X_val.reshape(-1, batch_size)
X_val = X_val.tolist()
y_val = y_val[:val_max_idx]
y_val = y_val.reshape(-1, batch_size)

X_test = X_test[:test_max_idx]
X_test = X_test.reshape(-1, batch_size)
X_test = X_test.tolist()
y_test = y_test[:test_max_idx]
y_test = y_test.reshape(-1, batch_size)

print("Batches Generated: ")
print("X_train: ", len(X_train))
print("y_train: ", y_train.shape)

print("X_val: ", len(X_val))
print("y_val: ", y_val.shape)

Batches Generated: 
X_train:  62611
y_train:  (62611, 16)
X_val:  14672
y_val:  (14672, 16)


In [8]:
# Model Training
from transformers.utils import logging
logging.set_verbosity(40)

from tqdm import tqdm
model = model.to(device)

num_epochs = 2
for epoch in range(num_epochs):
  model.train()
  for text, labels in tqdm(zip(X_train, y_train), total=len(X_train)):
    # prepare model input through our tokenizer
    model_inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=256)
    # place everything on the right device
    model_inputs = {k:v.to(device) for k,v in model_inputs.items()}
    # labels have to be torch long tensors
    labels = torch.tensor(labels).long()
    labels = labels.to(device)
    # now, we can perform the forward pass
    output = model(**model_inputs, labels=labels)
    loss, logits = output[:2]
    # and the backward pass
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

torch.save(model, dbert_model_path)
print("Model Saved")

100%|██████████| 62611/62611 [1:47:16<00:00,  9.73it/s]


Model Saved


In [9]:
# Predictions on validation dataset
print("Predicting on Validation Data")
loaded_model = torch.load(dbert_model_path)
predictions, targets = [], []
pred_logits = []
loaded_model.eval()

with torch.no_grad():
  for text, labels in tqdm(zip(X_val, y_val), total=len(X_val)):
    try:
      model_inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
      model_inputs = {k:v.to(device) for k,v in model_inputs.items()}

      output = loaded_model(**model_inputs)
      logits = output[0]
      pred_logits.append(logits)
      # prediction is the argmax of the logits
      predictions.extend(logits.argmax(dim=1).tolist())
      targets.extend(labels)
    except:
      print("Unable to process: ", text)
      continue

accuracy = metrics.accuracy_score(targets, predictions)
print ("accuracy", accuracy)
classification_report = metrics.classification_report(targets, predictions)
print (classification_report)

Predicting on Validation Data


100%|██████████| 14672/14672 [13:29<00:00, 18.13it/s]


accuracy 0.8907101962922573
              precision    recall  f1-score   support

           0       0.58      0.05      0.09     26001
           1       0.89      1.00      0.94    208751

    accuracy                           0.89    234752
   macro avg       0.74      0.52      0.51    234752
weighted avg       0.86      0.89      0.85    234752



In [9]:
# Predictions on Test data
print("Predicting on Test Data")
loaded_model = torch.load(dbert_model_path)
predictions, targets = [], []
pred_logits = []
loaded_model.eval()

with torch.no_grad():
  for text, labels in tqdm(zip(X_test, y_test), total=len(X_test)):
    try:
      model_inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
      model_inputs = {k:v.to(device) for k,v in model_inputs.items()}

      output = loaded_model(**model_inputs)
      logits = output[0]
      pred_logits.append(logits)
      # prediction is the argmax of the logits
      predictions.extend(logits.argmax(dim=1).tolist())
      targets.extend(labels)
    except:
      print("Unable to process: ", text)
      continue

from sklearn import metrics
accuracy = metrics.accuracy_score(targets, predictions)
print ("accuracy", accuracy)
classification_report = metrics.classification_report(targets, predictions)
print (classification_report)

Predicting on Test Data


100%|██████████| 16551/16551 [16:02<00:00, 17.20it/s]


accuracy 0.8427021025919884
              precision    recall  f1-score   support

           0       0.65      0.03      0.07     42333
           1       0.84      1.00      0.91    222483

    accuracy                           0.84    264816
   macro avg       0.75      0.52      0.49    264816
weighted avg       0.81      0.84      0.78    264816



In [ ]:
from google.colab import runtime
runtime.unassign()